# Révision de Croyances et Incohérence

**Navigation**: [← Tweety-3-Advanced-Logics](Tweety-3-Advanced-Logics.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-5-Abstract-Argumentation →](Tweety-5-Abstract-Argumentation.ipynb)

---

## Objectifs pédagogiques

1. Comprendre les principes AGM de révision de croyances
2. Explorer les mesures d'incohérence sur les bases de connaissances
3. Maîtriser l'énumération de MUS (Minimal Unsatisfiable Subsets)
4. Découvrir MaxSAT pour la satisfaction partielle

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Limitations connues (Tweety 1.28):**
> - Les imports CrMas (Credibility-based Multi-Agent Systems) peuvent échouer suite à une refactorisation de l'API
> - Les sections 3.2-3.4 (Mesures d'Incohérence, MUS, MaxSAT) fonctionnent correctement

In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration des outils externes (standalone) ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    # Also check if it's a directory (for Tweety compatibility)
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# Auto-detection Clingo (binaire)
# IMPORTANT: TweetyProject ClingoSolver expects a DIRECTORY path, not the full executable path.
# It appends "/clingo" (Unix) or "/clingo.exe" (Windows) internally when invoking the solver.
clingo_paths = [
    shutil.which("clingo"),
    shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/clingo/clingo.exe"),
    pathlib.Path("../ext_tools/clingo/clingo.exe"),
]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str):
            # For system PATH clingo, use the directory containing it
            cp_path = pathlib.Path(cp)
            EXTERNAL_TOOLS["CLINGO"] = str(cp_path.parent.resolve())
            break
        elif cp.exists():
            # For local clingo.exe, store the DIRECTORY path (parent)
            EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve())
            break

# Auto-detection SPASS
spass_paths = [
    shutil.which("SPASS"),
    shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/spass/SPASS.exe"),
    pathlib.Path("../ext_tools/spass/SPASS.exe"),
]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str):
            EXTERNAL_TOOLS["SPASS"] = sp
            break
        elif sp.exists():
            EXTERNAL_TOOLS["SPASS"] = str(sp.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists():
        jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
        print("JVM non disponible.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools:
        print(f"Outils externes: {', '.join(tools)}")

## Partie 3 : Révision de Croyances et Analyse d'Incohérence
<a id="partie3"></a>

Cette partie aborde des mécanismes de raisonnement plus avancés : comment mettre à jour des croyances face à de nouvelles informations (révision) et comment quantifier ou gérer les contradictions (incohérence). Nous nous concentrerons principalement sur la logique propositionnelle pour ces exemples.

### 3.1 Révision de Croyances Multi-Agents (CrMas)
<a id="3.1"></a>

La révision de croyances s'intéresse à l'intégration de nouvelles informations dans une base de connaissances existante, en résolvant les éventuelles contradictions de manière rationnelle (cf. postulats AGM). Tweety implémente notamment des approches pour des bases de croyances **multi-agents** où chaque information est associée à un **agent** et où un **ordre de crédibilité** peut exister entre ces agents.

* **`CrMasBeliefSet`** : Base de croyances multi-agents, nécessite un `Order<Agent>`.
* **`InformationObject`** : Encapsule une formule et l'agent source.
* **Opérateurs de Révision** (`org.tweetyproject.beliefdynamics.mas`) :
    * `CrMasRevisionWrapper`: Utilise un opérateur de révision classique (ex: Levi) sur les formules, en ignorant la structure multi-agents sauf pour prioriser la nouvelle information.
    * `CrMasSimpleRevisionOperator`: Prend en compte la crédibilité des agents de manière simple pour fusionner les informations.
    * `CrMasArgumentativeRevisionOperator`: Modélise la révision comme un processus argumentatif où les informations des agents plus crédibles peuvent "attaquer" celles des agents moins crédibles.

L'exemple suivant reprend la logique de `CrMasExample.java` et du TP C#.

In [ ]:
# --- 3.1 Revision de Croyances Multi-Agents (CrMas) ---
# NOTE: Cette section utilise des classes qui ont change de package dans Tweety 1.28.
# Chemins corriges pour Tweety 1.28:
#   - InformationObject: org.tweetyproject.beliefdynamics.mas.InformationObject
#   - CrMasSimpleRevisionOperator: org.tweetyproject.beliefdynamics.operators
#   - CrMasArgumentativeRevisionOperator: org.tweetyproject.beliefdynamics.operators
#   - LeviMultipleBaseRevisionOperator: org.tweetyproject.beliefdynamics (racine)
#   - DefaultMultipleBaseExpansionOperator: org.tweetyproject.beliefdynamics (racine)

print("\n--- 3.1 Revision de Croyances Multi-Agents (CrMas) ---")

CrMas_Imports_OK = False

if not jvm_ready:
    print("ERREUR: JVM non demarree. Impossible de continuer cet exemple.")
else:
    print("JVM prete. Tentative de chargement des classes CrMas...")
    try:
        import jpype
        from jpype.types import *

        # === Test des imports CrMas avec les chemins corrects pour Tweety 1.28 ===
        missing_imports = []

        # InformationObject est maintenant dans mas (pas racine beliefdynamics)
        try:
            InformationObject = jpype.JClass("org.tweetyproject.beliefdynamics.mas.InformationObject")
            print("   OK: InformationObject (from beliefdynamics.mas)")
        except Exception as e:
            print(f"   FAIL: InformationObject - {e}")
            missing_imports.append("InformationObject")

        # CrMasBeliefSet
        try:
            CrMasBeliefSet = jpype.JClass("org.tweetyproject.beliefdynamics.mas.CrMasBeliefSet")
            print("   OK: CrMasBeliefSet")
        except Exception as e:
            print(f"   FAIL: CrMasBeliefSet - {e}")
            missing_imports.append("CrMasBeliefSet")

        # CrMasRevisionWrapper
        try:
            CrMasRevisionWrapper = jpype.JClass("org.tweetyproject.beliefdynamics.mas.CrMasRevisionWrapper")
            print("   OK: CrMasRevisionWrapper")
        except Exception as e:
            print(f"   FAIL: CrMasRevisionWrapper - {e}")
            missing_imports.append("CrMasRevisionWrapper")

        # CrMasSimpleRevisionOperator est dans operators (pas mas)
        try:
            CrMasSimpleRevisionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.operators.CrMasSimpleRevisionOperator")
            print("   OK: CrMasSimpleRevisionOperator (from operators)")
        except Exception as e:
            print(f"   FAIL: CrMasSimpleRevisionOperator - {e}")
            missing_imports.append("CrMasSimpleRevisionOperator")

        # CrMasArgumentativeRevisionOperator est dans operators (pas mas)
        try:
            CrMasArgumentativeRevisionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.operators.CrMasArgumentativeRevisionOperator")
            print("   OK: CrMasArgumentativeRevisionOperator (from operators)")
        except Exception as e:
            print(f"   FAIL: CrMasArgumentativeRevisionOperator - {e}")
            missing_imports.append("CrMasArgumentativeRevisionOperator")

        # LeviMultipleBaseRevisionOperator est dans racine beliefdynamics (pas operators)
        try:
            LeviMultipleBaseRevisionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.LeviMultipleBaseRevisionOperator")
            print("   OK: LeviMultipleBaseRevisionOperator (from beliefdynamics)")
        except Exception as e:
            print(f"   FAIL: LeviMultipleBaseRevisionOperator - {e}")
            missing_imports.append("LeviMultipleBaseRevisionOperator")

        # DefaultMultipleBaseExpansionOperator est dans racine beliefdynamics (pas operators)
        try:
            DefaultMultipleBaseExpansionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.DefaultMultipleBaseExpansionOperator")
            print("   OK: DefaultMultipleBaseExpansionOperator (from beliefdynamics)")
        except Exception as e:
            print(f"   FAIL: DefaultMultipleBaseExpansionOperator - {e}")
            missing_imports.append("DefaultMultipleBaseExpansionOperator")

        # KernelContractionOperator et RandomIncisionFunction
        try:
            KernelContractionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.kernels.KernelContractionOperator")
            RandomIncisionFunction = jpype.JClass("org.tweetyproject.beliefdynamics.kernels.RandomIncisionFunction")
            print("   OK: KernelContractionOperator, RandomIncisionFunction")
        except Exception as e:
            print(f"   FAIL: Kernel classes - {e}")
            missing_imports.append("KernelClasses")

        # Agent classes
        try:
            Agent = jpype.JClass("org.tweetyproject.agents.Agent")
            DummyAgent = jpype.JClass("org.tweetyproject.agents.DummyAgent")
            Order = jpype.JClass("org.tweetyproject.comparator.Order")
            print("   OK: Agent, DummyAgent, Order")
        except Exception as e:
            print(f"   FAIL: Agent classes - {e}")
            missing_imports.append("AgentClasses")

        if not missing_imports:
            CrMas_Imports_OK = True
            print("\n==> Tous les imports CrMas reussis!")
        else:
            print(f"\n==> Imports CrMas echoues: {missing_imports}")
            print("   Note: Certaines classes peuvent avoir change de package dans Tweety 1.28")

    except Exception as e:
        print(f"ERREUR lors du test des imports CrMas: {e}")

print(f"\nCrMas_Imports_OK = {CrMas_Imports_OK}")

# === Execution de l'exemple CrMas si imports OK ===
if CrMas_Imports_OK:
    print("\n--- Execution de l'exemple CrMas ---")
    try:
        # Imports Java util et PL
        from java.util import ArrayList, HashSet, Collection
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import PlFormula, PlSignature
        from org.tweetyproject.logics.pl.reasoner import SimplePlReasoner

        # Classes Java necessaires pour les casts
        PlFormula_class = jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula")
        Agent_class = jpype.JClass("org.tweetyproject.agents.Agent")
        KernelProvider_class = jpype.JClass("org.tweetyproject.commons.KernelProvider")
        Collection_class = jpype.JClass("java.util.Collection")

        # --- Initialisation ---
        parser = PlParser()
        agents_list = ArrayList()
        agent1 = DummyAgent("A1"); agent2 = DummyAgent("A2"); agent3 = DummyAgent("A3")
        agents_list.add(agent1); agents_list.add(agent2); agents_list.add(agent3)

        # Ordre de credibilite: A1 > A2 > A3
        credOrder = Order(JObject(agents_list, Collection_class))
        credOrder.setOrderedBefore(agent1, agent2)
        credOrder.setOrderedBefore(agent2, agent3)
        print("\nOrdre de credibilite: A1 > A2 > A3")

        # Base de croyances initiale
        pl_sig_empty = PlSignature()
        base = CrMasBeliefSet(credOrder, pl_sig_empty)

        # Ajout InformationObject
        info1 = InformationObject(JObject(parser.parseFormula("!c"), PlFormula_class), JObject(agent2, Agent_class))
        info2 = InformationObject(JObject(parser.parseFormula("b"), PlFormula_class), JObject(agent3, Agent_class))
        info3 = InformationObject(JObject(parser.parseFormula("!b||!a"), PlFormula_class), JObject(agent3, Agent_class))
        base.add(info1); base.add(info2); base.add(info3)
        print(f"Base apres ajouts: {base}")

        # Nouvelles informations
        news_collection = HashSet()
        news_collection.add(InformationObject(JObject(parser.parseFormula("a"), PlFormula_class), JObject(agent3, Agent_class)))
        news_collection.add(InformationObject(JObject(parser.parseFormula("!a||c"), PlFormula_class), JObject(agent3, Agent_class)))

        news_str = ", ".join([str(info.getFormula())+" ("+str(info.getSource().getName())+")" for info in news_collection])
        print(f"\nNouvelles infos: {{ {news_str} }}")
        print(f"\nCalcul des revisions pour : {base} * {{ {news_str} }}")

        # --- Operateurs de Revision ---

        # 1) Revision priorisee simple (Levi Wrapper)
        print("\n1. Revision Priorisee Simple (Levi Wrapper):")
        try:
            incision_func = RandomIncisionFunction()
            pl_reasoner_instance = SimplePlReasoner()
            # Note: KernelContractionOperator attend un KernelProvider, SimplePlReasoner l'implemente
            kernel_contract = KernelContractionOperator(incision_func, JObject(pl_reasoner_instance, KernelProvider_class))
            levi_operator = LeviMultipleBaseRevisionOperator(kernel_contract, DefaultMultipleBaseExpansionOperator())
            revision_prio = CrMasRevisionWrapper(levi_operator)
            result_prio = revision_prio.revise(base, JObject(news_collection, Collection_class))
            print(f"   Resultat PRIO       : {result_prio}")
        except jpype.JException as e_prio:
             print(f"   Erreur Java (PRIO): {e_prio.message()}")
        except Exception as e_prio_py:
             print(f"   Erreur Python (PRIO): {e_prio_py}")

        # 2) Revision non priorisee simple (Credibilite)
        print("\n2. Revision Non-Priorisee Simple (Credibilite):")
        try:
            revision_nonprio = CrMasSimpleRevisionOperator()
            result_nonprio = revision_nonprio.revise(base, JObject(news_collection, Collection_class))
            print(f"   Resultat N-PRIO CRED: {result_nonprio}")
        except jpype.JException as e_nonprio:
             print(f"   Erreur Java (N-PRIO): {e_nonprio.message()}")
        except Exception as e_nonprio_py:
             print(f"   Erreur Python (N-PRIO): {e_nonprio_py}")

        # 3) Revision argumentative (Credibilite)
        print("\n3. Revision Argumentative (Credibilite):")
        try:
            revision_arg = CrMasArgumentativeRevisionOperator()
            result_arg = revision_arg.revise(base, JObject(news_collection, Collection_class))
            print(f"   Resultat ARG        : {result_arg}")
        except jpype.JException as e_arg:
             print(f"   Erreur Java (ARG): {e_arg.message()}")
        except Exception as e_arg_py:
             print(f"   Erreur Python (ARG): {e_arg_py}")

    except ImportError as e:
        print(f"Erreur d'import initiale pour CrMas : {e}")
    except jpype.JException as e_java:
        print(f"Erreur Java generale dans l'exemple CrMas: {e_java.message()}")
    except Exception as e_gen:
        print(f"Erreur Python inattendue dans l'exemple CrMas: {e_gen}")
        import traceback
        traceback.print_exc()
else:
    print("\nExecution CrMas ignoree car imports echoues.")

### 3.2 Mesures d'Incohérence (PL)
<a id="3.2"></a>

Mesurer le degré d'incohérence d'une base de connaissances propositionnelle.

* **`ContensionInconsistencyMeasure`**: Basée sur le nombre minimal de variables à assigner pour trouver un modèle.
* **`MaInconsistencyMeasure` / `McscInconsistencyMeasure`**: Basées sur les MUS (Minimal Unsatisfiable Subsets). Nécessitent un énumérateur de MUS (ex: `MarcoMusEnumerator` externe ou `NaiveMusEnumerator` interne).
* **`FuzzyInconsistencyMeasure`**: Utilise une sémantique floue pour évaluer la satisfaction des formules.
* **`DSum/DMax/DHitInconsistencyMeasure`**: Basées sur la distance (ex: distance de Dalal) entre la base et les mondes possibles les plus proches la satisfaisant.

In [ ]:
# --- 3.2 Mesures d'Incoherence (PL) ---
# NOTE: PlParser est dans org.tweetyproject.logics.pl.parser (pas .syntax)
# Les mesures de distance ont ete renommees en Tweety 1.28:
#   - DSumInconsistencyMeasure -> DSumSatInconsistencyMeasure
#   - DMaxInconsistencyMeasure -> DMaxSatInconsistencyMeasure
#   - DHitInconsistencyMeasure -> DHitSatInconsistencyMeasure
# Ma/Mcsc sont dans org.tweetyproject.logics.commons.analysis
# FuzzyInconsistencyMeasure necessite un solveur non-lineaire (non disponible par defaut)

print("\n--- 3.2 Mesures d'Incoherence (PL) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution des exemples de mesures d'incoherence...")
    try:
        import jpype
        from jpype.types import *

        # Imports PL de base - PlParser est dans .parser, pas .syntax!
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import PlBeliefSet, PlFormula, PlSignature
        from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver
        from java.util import Collection
        print("   OK: Imports PL de base (PlParser from .parser)")

        # Imports pour le solveur d'optimisation
        from org.tweetyproject.math.opt.solver import Solver, ApacheCommonsSimplex
        Solver.setDefaultGeneralSolver(ApacheCommonsSimplex())
        print("   OK: Solveur d'optimisation configure (ApacheCommonsSimplex)")

        # Imports specifiques aux mesures
        from org.tweetyproject.logics.pl.analysis import ContensionInconsistencyMeasure
        # Note: DSum/DMax/DHit renommes en *Sat* dans Tweety 1.28
        from org.tweetyproject.logics.pl.analysis import DSumSatInconsistencyMeasure, DMaxSatInconsistencyMeasure, DHitSatInconsistencyMeasure
        from org.tweetyproject.logics.commons.analysis import NaiveMusEnumerator
        # Ma/Mcsc sont dans commons.analysis
        from org.tweetyproject.logics.commons.analysis import MaInconsistencyMeasure, McscInconsistencyMeasure

        print("   OK: Tous les imports pour Mesures d'Incoherence reussis.")

        # Classe pour cast
        Collection_class = jpype.JClass("java.util.Collection")

        # --- Configuration ---
        parser_inc = PlParser()
        SatSolver.setDefaultSolver(Sat4jSolver())

        # --- Exemples ---

        # 1. Contension (Base sur ContensionExample.java)
        print("\n--- Mesure Contension ---")
        kb_cont = PlBeliefSet()
        formulas_cont = ["a", "!a && b", "!b", "c || a", "!c || a", "!c || d", "!d", "d", "c"]
        for f_str in formulas_cont: kb_cont.add(parser_inc.parseFormula(f_str))
        print(f"KB: {kb_cont}")
        try:
            cont_measure = ContensionInconsistencyMeasure()
            # Cast en Collection pour eviter ambiguite JPype
            cont_value = cont_measure.inconsistencyMeasure(JObject(kb_cont, Collection_class))
            print(f"Valeur Contension: {cont_value}")
        except Exception as e_cont:
            print(f"Erreur Contension: {e_cont}")

        # 2. DSum / DMax / DHit (mesures basees sur distance de Dalal)
        print("\n--- Mesures Basees sur Distance (Dalal) ---")
        kb_dist = PlBeliefSet()
        kb_dist.add(parser_inc.parseFormula("a && b && c"))
        kb_dist.add(parser_inc.parseFormula("!a && !b && !c"))
        print(f"KB pour Distance: {kb_dist}")
        try:
            dsum_measure = DSumSatInconsistencyMeasure()
            dsum_value = dsum_measure.inconsistencyMeasure(JObject(kb_dist, Collection_class))
            print(f"Valeur DSum (Sat): {dsum_value}")

            dmax_measure = DMaxSatInconsistencyMeasure()
            dmax_value = dmax_measure.inconsistencyMeasure(JObject(kb_dist, Collection_class))
            print(f"Valeur DMax (Sat): {dmax_value}")

            dhit_measure = DHitSatInconsistencyMeasure()
            dhit_value = dhit_measure.inconsistencyMeasure(JObject(kb_dist, Collection_class))
            print(f"Valeur DHit (Sat): {dhit_value}")
        except jpype.JException as e_dist_java:
             print(f"Erreur Java (Mesures Distance): {e_dist_java.message()}")
        except Exception as e_dist_py:
             print(f"Erreur Python (Mesures Distance): {e_dist_py}")

        # 3. Mesures Ma et Mcsc (basees sur MUS)
        print("\n--- Mesures Ma / Mcsc (basees sur MUS) ---")
        kb_mus_demo = PlBeliefSet()
        formulas_mus_demo = ["a", "!a", "!a && !b", "b"]
        for f_str in formulas_mus_demo: kb_mus_demo.add(parser_inc.parseFormula(f_str))
        print(f"KB pour Ma/Mcsc: {kb_mus_demo}")
        try:
            mus_enum_naive = NaiveMusEnumerator(SatSolver.getDefaultSolver())
            ma_measure_naive = MaInconsistencyMeasure(mus_enum_naive)
            ma_value = ma_measure_naive.inconsistencyMeasure(JObject(kb_mus_demo, Collection_class))
            print(f"Valeur Ma (Naive): {ma_value}")

            mcsc_measure_naive = McscInconsistencyMeasure(mus_enum_naive)
            mcsc_value = mcsc_measure_naive.inconsistencyMeasure(JObject(kb_mus_demo, Collection_class))
            print(f"Valeur Mcsc (Naive): {mcsc_value}")
        except Exception as e_ma_mcsc:
             print(f"Erreur calcul Ma/Mcsc (Naive): {e_ma_mcsc}")

        # Note: FuzzyInconsistencyMeasure necessite un solveur non-lineaire
        print("\n--- Note: FuzzyInconsistencyMeasure ---")
        print("FuzzyInconsistencyMeasure necessite un solveur d'optimisation non-lineaire")
        print("(pas disponible par defaut). Omis dans cet exemple.")

    except ImportError as e:
        print(f"Erreur d'import pour Mesures d'Incoherence : {e}")
    except jpype.JException as e_java:
        print(f"Erreur Java generale dans Mesures d'Incoherence: {e_java.message()}")
    except Exception as e_gen:
        print(f"Erreur Python inattendue dans Mesures d'Incoherence: {e_gen}")
        import traceback; traceback.print_exc()

### 3.3 Énumération de MUS (Minimal Unsatisfiable Subsets)
<a id="3.3"></a>

Un Sous-ensemble Minimal Inconsistant (MUS) d'une base de connaissances $KB$ est un sous-ensemble $M \subseteq KB$ tel que $M$ est inconsistant, mais tout sous-ensemble propre de $M$ est consistant. Trouver les MUS est utile pour diagnostiquer les sources d'incohérence.

* **`NaiveMusEnumerator`**: Implémentation simple mais potentiellement très lente, intégrée à Tweety.
* **`MarcoMusEnumerator`**: Interface avec l'outil externe `marco.py`, beaucoup plus efficace. Nécessite d'installer MARCO et de fournir le chemin vers `marco.py`.

In [ ]:
# --- 3.3 Enumeration de MUS ---
# MUS = Minimal Unsatisfiable Subsets
# Utile pour diagnostiquer les sources d'incoherence dans une base de connaissances

print("\n--- 3.3 Enumeration de MUS (Minimal Unsatisfiable Subsets) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple MUS...")
    try:
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import PlBeliefSet
        from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver
        from org.tweetyproject.logics.commons.analysis import NaiveMusEnumerator

        # --- Configuration ---
        parser_mus = PlParser()
        SatSolver.setDefaultSolver(Sat4jSolver())

        # Exemple simple (MusExample.java simplifie)
        kb_mus_ex = PlBeliefSet()
        formulas_mus_ex = ["a", "!a", "!a && !b", "b", "c", "!c", "!a || !c"]
        for f_str in formulas_mus_ex: kb_mus_ex.add(parser_mus.parseFormula(f_str))

        print("KB pour MUS:")
        for f in kb_mus_ex:
            print(f"  - {f}")

        # --- Enumeration ---

        # Option 1: NaiveMusEnumerator (integre a Tweety)
        print("\nCalcul des MUS (Naive Enumerator - peut etre lent sur grandes KB):")
        try:
            mus_enum_naive = NaiveMusEnumerator(SatSolver.getDefaultSolver())
            all_mus_naive = mus_enum_naive.minimalInconsistentSubsets(kb_mus_ex)
            print(f" - Trouve {len(all_mus_naive)} MUS:")
            for mus in all_mus_naive:
                mus_str = ", ".join([str(f) for f in mus])
                print(f"   - {{ {mus_str} }}")
        except Exception as e:
            print(f"Erreur NaiveMusEnumerator: {e}")

        # Option 2: MarcoMusEnumerator (Externe)
        # Necessite l'installation de MARCO (https://github.com/Z3Prover/z3/tree/master/src/api/python/z3/z3consts.py)
        # et la configuration du chemin vers marco.py
        # Decommentez le code ci-dessous si MARCO est configure:
        #
        # from org.tweetyproject.logics.pl.sat import MarcoMusEnumerator
        # MARCO_PATH = "/path/to/your/marco.py"  # <--- MODIFIEZ CECI
        # if pathlib.Path(MARCO_PATH).exists():
        #     print("\nCalcul des MUS (Marco Enumerator - plus efficace):")
        #     try:
        #         mus_enum_marco = MarcoMusEnumerator(MARCO_PATH)
        #         all_mus_marco = mus_enum_marco.minimalInconsistentSubsets(kb_mus_ex)
        #         print(f" - Trouve {len(all_mus_marco)} MUS:")
        #         for mus in all_mus_marco:
        #             print(f"   - {mus}")
        #     except Exception as e:
        #         print(f"Erreur MarcoMusEnumerator: {e}")
        # else:
        #     print("\nMarco MUS Enumerator non trouve/configure.")

    except ImportError as e:
        print(f"Erreur d'import pour MUS: {e}")
    except jpype.JException as e_java:
        print(f"Erreur Java dans MUS: {e_java.message()}")
    except Exception as e_gen:
        print(f"Erreur Python inattendue dans MUS: {e_gen}")
        import traceback; traceback.print_exc()

### 3.4 MaxSAT
<a id="3.4"></a>

MaxSAT (Maximum Satisfiability) est une généralisation du problème SAT. Étant donné un ensemble de clauses "dures" (qui doivent être satisfaites) et un ensemble de clauses "molles" (qui peuvent être violées, souvent avec un coût/poids associé), MaxSAT cherche une assignation qui satisfait toutes les clauses dures et minimise le coût total des clauses molles violées (ou maximise le poids des clauses molles satisfaites).

Tweety intègre des solveurs MaxSAT externes comme **Open-WBO**.

* **`MaxSatSolver`**: Interface abstraite.
* **`OpenWboSolver`**: Implémentation pour Open-WBO (nécessite chemin).
* **Entrée**: Une `PlBeliefSet` pour les clauses dures, et une `Map<PlFormula, Integer>` pour les clauses molles et leurs poids (coûts de violation).
* **Sortie**: Une `Interpretation` (un `PossibleWorld`) qui est une solution optimale.

In [ ]:
# --- 3.4 MaxSAT ---
# MaxSAT = Maximum Satisfiability
# Trouve une assignation qui satisfait toutes les clauses dures
# et minimise le cout des clauses molles violees.
# Necessite un solveur MaxSAT externe comme Open-WBO.

print("\n--- 3.4 MaxSAT ---")
print("Note: Cette section necessite l'installation d'un solveur MaxSAT externe (Open-WBO).")
print("      Le code ci-dessous est fourni a titre d'exemple et necessite la configuration")
print("      du chemin vers l'executable Open-WBO.")

if not jvm_ready:
    print("\nERREUR: JVM non demarree.")
else:
    try:
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import PlBeliefSet
        from java.util import HashMap

        # --- Configuration ---
        parser_maxsat = PlParser()

        # Clauses Dures (doivent etre satisfaites)
        hard_clauses_bs = PlBeliefSet()
        hard_formulas = ["!a && b", "b || c", "c || d", "f || (c && g)"]
        for f_str in hard_formulas: hard_clauses_bs.add(parser_maxsat.parseFormula(f_str))

        # Clauses Molles (avec poids/cout de violation)
        # Utiliser HashMap Java: Map<PlFormula, Integer>
        soft_clauses_map = HashMap()
        soft_clauses_map.put(parser_maxsat.parseFormula("a || !b"), 25)  # violer coute 25
        soft_clauses_map.put(parser_maxsat.parseFormula("!c"), 15)       # violer coute 15

        print("\nClauses Dures (doivent etre satisfaites):")
        for f in hard_clauses_bs:
            print(f"  - {f}")

        print("\nClauses Molles (avec poids de violation):")
        for entry in soft_clauses_map.entrySet():
            print(f"  - {entry.getKey()} (poids: {entry.getValue()})")

        # --- Resolution (necessite OpenWBO) ---
        # Pour utiliser Open-WBO:
        # 1. Telecharger Open-WBO: https://github.com/sat-group/open-wbo
        # 2. Compiler l'executable
        # 3. Configurer OPENWBO_PATH ci-dessous
        #
        # OPENWBO_PATH = "/path/to/your/open-wbo_exec"  # <--- MODIFIEZ CECI
        # if pathlib.Path(OPENWBO_PATH).exists():
        #     from org.tweetyproject.logics.pl.sat import OpenWboSolver
        #     print("\nResolution MaxSAT (OpenWBO):")
        #     try:
        #         maxsat_solver = OpenWboSolver(OPENWBO_PATH)
        #         witness_maxsat = maxsat_solver.getWitness(hard_clauses_bs, soft_clauses_map)
        #
        #         if witness_maxsat:
        #             print(" - Solution (Interpretation):", witness_maxsat)
        #             cost = maxsat_solver.costOf(witness_maxsat, hard_clauses_bs, soft_clauses_map)
        #             print(f" - Cout de la solution (clauses molles violees * poids): {cost}")
        #             print(f"   - Les clauses dures sont satisfaites? {witness_maxsat.satisfies(hard_clauses_bs)}")
        #         else:
        #             print(" - Aucune solution trouvee (clauses dures potentiellement inconsistantes).")
        #
        #     except Exception as e:
        #          print(f"Erreur OpenWboSolver: {e}")
        # else:
        #      print("\nOpen-WBO non trouve/configure, test MaxSAT externe saute.")

        print("\n(Pour executer ce test, installez Open-WBO et decommentez le code ci-dessus)")

    except ImportError as e:
        print(f"Erreur d'import pour MaxSAT: {e}")
    except Exception as e_gen:
        print(f"Erreur: {e_gen}")

---

## Résumé

Ce notebook a couvert:
- **Révision de Croyances (CrMas)**: Principes AGM, multi-agents (limitation API 1.28)
- **Mesures d'Incohérence**: Quantification du degré d'incohérence d'une KB
- **MUS**: Identification des sous-ensembles minimaux insatisfiables
- **MaxSAT**: Satisfaction maximale avec contraintes partielles

## Prochaines étapes

Le notebook suivant introduit l'argumentation abstraite de Dung avec ses différentes sémantiques.

---

**Navigation**: [← Tweety-3-Advanced-Logics](Tweety-3-Advanced-Logics.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-5-Abstract-Argumentation →](Tweety-5-Abstract-Argumentation.ipynb)